### Code yang diperlukan

In [84]:
# split data to 14001 file, reference: https://stackoverflow.com/questions/31786287/how-to-split-large-text-file-in-windows

import os
import time
import datetime
import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import losses

from network_module import dense_block, transition_block, Conv1DTranspose

from tensorflow.python.client import device_lib
print (device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16515514961516078718
xla_global_id: -1
]


In [93]:
TOTAL_DATA = 14001  # Hardcode jumlah data training
BATCH_SIZE = 2  # Hardcode batch size
# wifi_train_data = np.loadtxt("data0601_LPS_2/train/w_train_data.txt").reshape(14001, 21, 15)
# mag_train_data = np.loadtxt("data0601_LPS_2/train/m_train_data.txt").reshape(14001, 21, 2)
# gps_train_data = np.loadtxt("data0601_LPS_2/train/g_train_data.txt").reshape(14001, 21, 4)
# tr_labelc = np.loadtxt("data0601_LPS_2/train/train_labelc.txt").reshape(14001, 21, 2)
# tr_label = np.loadtxt("data0601_LPS_2/train/train_label.txt").reshape(14001, 21)
# tr_wgs_label = np.loadtxt("data0601_LPS_2/train/wgs_label.txt").reshape(14001, 21, 15)

In [3]:
def load_gps_model(path):
    inp = tf.keras.layers.Input(shape=(4), name='input_layer1')
    model = tf.keras.layers.Reshape((4,1))(inp)
    model = tf.keras.layers.Conv1D(32,3)(model)
    model = tf.keras.layers.AveragePooling1D(strides=1,padding="same")(model)
    model = dense_block(32,3,3,4)(model)
    model = transition_block(stride=1)(model)
    model = dense_block(32,3,3,4)(model)
    model_down_gps = tf.keras.Model(inputs=[inp], outputs=model)
    # load gps trained model
    latest = tf.train.latest_checkpoint(path)

    checkpoint = tf.train.Checkpoint(model_down=model_down_gps)
    checkpoint.restore(latest).expect_partial()
    return model_down_gps

def load_wifi_model(path):
    inp = tf.keras.layers.Input(shape=(15), name='input_layer1')
    model = tf.keras.layers.Reshape((15,1))(inp)
    model = tf.keras.layers.Conv1D(32,3)(model)
    model = tf.keras.layers.AveragePooling1D(strides=1)(model)
    model = dense_block(32,3,3,4)(model)
    model = transition_block(stride=1)(model)
    model_down_wifi = tf.keras.Model(inputs=[inp], outputs=model)

    path = "./wifi_checkpoints/checkpoints_0524_conv/"
    latest = tf.train.latest_checkpoint(path)

    checkpoint = tf.train.Checkpoint(model_down=model_down_wifi)
    checkpoint.restore(latest).expect_partial()
    return model_down_wifi

def gps_preprocess(model_down_gps, gps_train_data):
    #print("gps preprocess")
    #memasukkan data ke dalam model
    GPS = model_down_gps(gps_train_data.astype('float32'))  # GPS = tf.keras.Model(inputs=[gps_train_data.astype('float32')], outputs=model)
    #reshape data menjadi data 2D dengan baris shape GPS.shape[0], kolomnya otomatis -> makanya pakai -1
    train_GPS = np.array(GPS).reshape(GPS.shape[0],-1)
    #print("gps shape : ", train_GPS[0].shape)

    #return data yang sudah dipre-process dan direshape -> sama dengan fungsi gps_preprocess
    return train_GPS

def wifi_preprocess(model_down_wifi, wifi_train_data):
    #print("wifi preprocess")
    wifi = model_down_wifi(wifi_train_data.astype('float32'))

    #reshape data menjadi data 2D dengan baris shape wifi.shape[0], kolomnya otomatis -> makanya pakai -1
    train_wifi = np.array(wifi).reshape(wifi.shape[0],-1)
    #print("wifi shape : ", train_wifi[0].shape)
    
    #return data yang sudah dipre-process dan direshape -> sama dengan fungsi wifi_preprocess
    return train_wifi

In [4]:
model_down_gps = load_gps_model(path="./gps_checkpoint-002/gps_checkpoints/checkpoints_0601_LPS_2/")
model_down_wifi = load_wifi_model(path="./wifi_checkpoints/checkpoints_0524_conv/")

In [24]:
def get_train_data():
    for i in range(TOTAL_DATA):
        train_data = []
        train_label = []
        train_iolabel = []
        train_wgs_label = []
        
        # asumsi start data dari 0
        wifi_train_data = np.loadtxt(f"./data0601_LPS_2/train/w_train_data/w_train_{str(i).zfill(5)}.txt").reshape(21, 15)
        wifi = wifi_preprocess(model_down_wifi, wifi_train_data)

        g_mag = np.loadtxt(f"data0601_LPS_2/train/m_train_data/m_train_{str(i).zfill(5)}.txt").reshape(21, 2)
        gps_train_data = np.loadtxt(f"./data0601_LPS_2/train/g_train_data/g_train_{str(i).zfill(5)}.txt").reshape(21, 4)
        GPS = gps_preprocess(model_down_gps, gps_train_data)
        lstm_input = np.concatenate([wifi, g_mag, GPS], 1)
        
        tr_label = np.loadtxt(f"./data0601_LPS_2/train/train_label/train_label_{str(i).zfill(5)}.txt").reshape(21,)
        tr_labelc = np.loadtxt(f"./data0601_LPS_2/train/train_labelc/train_labelc_{str(i).zfill(5)}.txt").reshape(21, 2)
        tr_wgs_label = np.loadtxt(f"./data0601_LPS_2/train/wgs_label/wgs_label_{str(i).zfill(5)}.txt").reshape(21, 15)
        
        for j in range(17):
            train_data.append(lstm_input[j:j+5])
            train_label.append(tr_labelc[j+4])
            if tr_label[j+4] < 26:
                train_iolabel.append(0)
            else:
                train_iolabel.append(1)
            train_wgs_label.append(np.array(tr_wgs_label[j:j+5]).reshape(-1))

        train_data = np.array(train_data).astype('float32')
        train_label = np.array(train_label).astype('float32')
        io_label = np.array(train_iolabel).astype('int32')
        wgs_label = np.array(train_wgs_label).astype('int32')
        io_label = tf.one_hot(io_label, 2)

        #print(train_data.shape, train_label.shape, io_label.shape, wgs_label.shape)
        yield (train_data, train_label, io_label, wgs_label)

In [61]:
def create_model():
    lstm_input = tf.keras.layers.Input(shape=(5,866), name='input_layer_lstm')
    lstm1 = tf.keras.layers.LSTM(20,return_sequences=True)(lstm_input)
    lstm2 = tf.keras.layers.LSTM(20,return_sequences=False)(lstm1)
    fc3 = tf.keras.layers.Dense(2,name='fc3')(lstm2)
    io3 = tf.keras.layers.Dense(2,activation='sigmoid',name='io3')(lstm2)
    wgs3 = tf.keras.layers.Dense(75,activation='sigmoid',name='wgs3')(lstm2)
    model_lstm = tf.keras.Model(inputs=[lstm_input], outputs=[fc3,io3,wgs3],name = "lstm")

    return model_lstm

In [63]:
model = create_model()
optimizer_A = tf.optimizers.Nadam()

In [64]:
def train_step(model, optimizer_A, t_x, t_y, t_io, t_wgs):
    lstm_vars = model.trainable_variables

    with tf.GradientTape() as lstm_tape, tf.GradientTape() as io_tape, tf.GradientTape() as wgs_tape:
        output = model(t_x, training=True)
        coor_loss, io_loss, wgs_loss = model_loss(output,t_y,t_io,t_wgs)
        coor_loss = tf.multiply(0.5,coor_loss)

    # rand = np.random.rand()
    # if rand < 0.4:
    gradients = lstm_tape.gradient([coor_loss,io_loss,wgs_loss], lstm_vars)# + categorial_vars)
    optimizer_A.apply_gradients(zip(gradients, lstm_vars))# + categorial_vars))

    return np.array(coor_loss).mean(),np.array(io_loss).mean(),np.array(wgs_loss).mean()

In [81]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

def model_loss(output, t_y, t_io, t_wgs):
    output_categorial , output_io, output_wgs = output
    coor_loss = euclidean_distance_loss(t_y,output_categorial)
    io_loss = losses.binary_crossentropy(t_io,output_io)
    wgs_loss = losses.binary_crossentropy(t_wgs,output_wgs)

    return coor_loss, io_loss, wgs_loss

In [99]:
def train(model, optimizer_A, epochs):
    # Lokasi weight
    checkpoint_dir = './lstm_checkpoints/AE_LSTM_checkpoints_0601_coor_LPS_2'
    # save weight berdasar checkpoint
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_dropout_model_{epoch}")
    checkpoint = tf.train.Checkpoint(optimizerA=optimizer_A, model_lstm=model)

    #save log
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = './tensorflow/logs/gradient_tape/' + current_time + '/train'
    test_log_dir = './tensorflow/logs/gradient_tape/' + current_time + '/test'
    # save log jadi file yang compatible dengan tensorboard sehingga bisa divisualisasikan
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)

    # berkaitan dengan lost function
    minimum_delta = 0.0001
    history = {}
    history['val_loss'] = np.zeros(epochs)

    dset = tf.data.Dataset.from_generator(get_train_data,
                                      (tf.float32,  # Data type for "train_data"
                                       tf.float32,  # Data type for "train_label"
                                       tf.int32,  # Data type for "io_label"
                                       tf.int32),  # Data type for "wgs_label"
                                     )
    
    for epoch in range(epochs):
        print("epoch : ", epoch)
        start = time.time()
        all_coor = []
        all_io = []
        all_wgs = []

        data = next(iter(dset.shuffle(5).batch(BATCH_SIZE)))

        train_data, train_label, io_label, wgs_label = data
        for h in range(BATCH_SIZE):
            for i in range(17):
                # Variable `data` in merupakan tuple (train_data, train_label, io_label, wgs_label)
                # Dimana masing-masing memiliki shape 4-dimensi, kita pakai contoh BATCH_SIZE = 1
                # Shape dari x/train_data == (1, 17, 5, 866)
                # Shape dari y/train_label == (1, 17, 2)
                # Shape dari io/io_label == (1, 17, 2)
                # Shape dari wgs/wgs_label == (1, 17, 75)

                x = tf.expand_dims(train_data[h][i], axis=0)  # Perlu expand_dims supaya seolah-olah data ini jadi memiliki shape (-1, 5, 866), jika tidak maka hanya akan menjadi (5, 866)
                y = tf.expand_dims(train_label[h][i], axis=0)
                io = tf.expand_dims(io_label[h][i], axis=0)
                wgs = tf.expand_dims(wgs_label[h][i], axis=0)
                coor_loss,io_loss,wgs_loss = train_step(model, optimizer_A, x,y,io,wgs)
                
                # print(x.shape)
                all_coor.append(coor_loss)
                all_io.append(io_loss)
                all_wgs.append(wgs_loss)
        
        print("train categorial loss : {}, train io loss : {}, train wgs loss : {}".format(np.array(all_coor).mean(),np.array(all_io).mean(),np.array(all_wgs).mean()))

        # train tulis ke file supaya bisa dibaca di tensorboard
        with train_summary_writer.as_default():
            tf.summary.scalar('coor loss', np.array(all_coor).mean(), step=epoch)
            tf.summary.scalar('io loss', np.array(all_io).mean(), step=epoch)
            tf.summary.scalar('wgs loss', np.array(all_wgs).mean(), step=epoch)

        # for (val_x, val_y, val_io, val_wgs) in self.test_dataset:
        #     loss, io_loss, wgs_loss = self.validation(val_x, val_y, val_io, val_wgs)
            # print("val end")
        
        # with test_summary_writer.as_default():
        #     tf.summary.scalar('coor loss', loss, step=epoch)
        #     tf.summary.scalar('io loss', io_loss, step=epoch)
        #     tf.summary.scalar('wgs loss', wgs_loss, step=epoch)

        # print("sum end")
        # history['val_loss'][epoch] = loss
        checkpoint.save(file_prefix=checkpoint_prefix)

        # if epoch > 200: #early stop
        #     differences = np.abs(np.diff(history['val_loss'][epoch - 5:epoch], n = 1))
        #     check =  differences > minimum_delta 

        #     if np.all(check == False):
        #         print(differences)
        #         print("\n\nEarlyStopping Evoked! Stopping training\n\n")
        #         break

In [100]:
train(model, optimizer_A, epochs=10)

epoch :  0
train categorial loss : 1.0012296438217163, train io loss : 0.3453819155693054, train wgs loss : 0.458695650100708
epoch :  1
train categorial loss : 1.012282371520996, train io loss : 0.3894517123699188, train wgs loss : 0.5302849411964417
epoch :  2
train categorial loss : 0.8670793175697327, train io loss : 0.30510184168815613, train wgs loss : 0.5015010237693787
epoch :  3
train categorial loss : 0.9131344556808472, train io loss : 0.22039832174777985, train wgs loss : 0.39947906136512756
epoch :  4
train categorial loss : 0.7445359230041504, train io loss : 0.18408595025539398, train wgs loss : 0.38137057423591614
epoch :  5
train categorial loss : 0.9210472702980042, train io loss : 0.29982325434684753, train wgs loss : 0.5165290832519531
epoch :  6
train categorial loss : 0.9136998653411865, train io loss : 0.18729694187641144, train wgs loss : 0.4023730158805847
epoch :  7
train categorial loss : 0.8477675914764404, train io loss : 0.15233919024467468, train wgs loss

In [ ]:
data = next(iter(dset.shuffle(5).batch(2)))
data

In [ ]:
data

In [ ]:
def get_train_data(batch_size):
    train_data = []
    train_label = []
    train_iolabel = []
    train_wgs_label = []

    for i in range(batch_size):
        wifi = train_wifi[i]
        g_mag = mag_train_data[i]
        GPS = train_GPS[i]
        lstm_input = np.concatenate([wifi, g_mag, GPS], 1)
        for j in range(17):
            train_data.append(lstm_input[j:j+5])
            train_label.append(tr_labelc[i][j+4])
            if tr_label[i][j+4] < 26:
                train_iolabel.append(0)
            else:
                train_iolabel.append(1)
            train_wgs_label.append(np.array(tr_wgs_label[i][j:j+5]).reshape(-1))

    train_data = np.array(train_data).astype('float32')
    train_label = np.array(train_label).astype('float32')
    io_label = np.array(train_iolabel).astype('int32')
    wgs_label = np.array(train_wgs_label).astype('int32')
    io_label = tf.one_hot(io_label, 2)

    print(train_data.shape, train_label.shape, io_label.shape, wgs_label.shape)
    yield (train_data, train_label, io_label, wgs_label)

In [ ]:
# Generate Tensorflow dataset object "dset"
dset = tf.data.Dataset.from_generator(get_train_data,
                                      (tf.float32,  # Data type for "train_data"
                                       tf.float32,  # Data type for "train_label"
                                       tf.int32,  # Data type for "io_label"
                                       tf.int32),  # Data type for "wgs_label"
                                      args=[BATCH_SIZE])

In [ ]:
for train_data, train_label, io_label, wgs_label in dset:
    print(train_data.shape, train_label.shape, io_label.shape, wgs_label.shape)

In [ ]:
for train_data, train_label, io_label, wgs_label in dset:
    print(f"train_data.numpy:\n{train_data.numpy()}")
    print(f"train_label.numpy:\n{train_label.numpy()}")
    print(f"io_label.numpy:\n{io_label.numpy()}")
    print(f"wgs_label.numpy:\n{wgs_label.numpy()}")

### Testing, mohon abaikan

In [ ]:
def count(stop):
    i = 0
    while i<stop:
        yield i
        i += 1

In [ ]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes = ())

In [ ]:
# Batch: panjang per data yang diambil, misal batch(10) berarti per data panjangnya 10
# Take: berapa banyak data yang diambil, misal take(10) berarti ambil 10 data dari generator
for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())

In [ ]:
for count_batch in ds_counter.repeat().batch(2).take(10):
    print(count_batch.numpy())